#**. Executar Extração**

##**. Instalação**

In [ ]:
# @title Instalação
!pip install pandas
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##**. Setup**

In [ ]:
# @title Setup

from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
from google.cloud import storage
import pandas as pd
import os

project = 'athenas-364914' 
location = 'US' 
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

##**. Visualização de Querry**

In [ ]:
# @title SQL de consulta

job = client.get_job('bquxjob_748b07d6_183c4030139') 
print(job.query)

SELECT
  `athenas-364914.Athenas2.Ligacoes_dist`.midia AS TIPO_MIDIA,
  FORMAT_DATETIME('%m/%y',`athenas-364914.Athenas2.Ligacoes_dist`.created_at)AS MES_ANO,
  COUNT(`athenas-364914.Athenas2.Ligacoes_dist`.status_id) AS QUANTIDADE
FROM
  `athenas-364914.Athenas2.Ligacoes_dist`
WHERE
  `athenas-364914.Athenas2.Ligacoes_dist`.status_id NOT IN(57,
    100)
GROUP BY
  MES_ANO,
  TIPO_MIDIA;


##**. Visualização de Dados**

In [ ]:
# @title Visualização

job = client.get_job('bquxjob_748b07d6_183c4030139') 
results = job.to_dataframe()
results

,TIPO_MIDIA,MES_ANO,QUANTIDADE
0,CPC,None,3160
1,ORGANIC,None,9965
2,SMS,None,660
3,ORGANIC,06/20,420
4,ORGANIC,09/20,812
5,ORGANIC,01/20,915
6,ORGANIC,05/20,580
7,ORGANIC,12/20,533
8,ORGANIC,08/20,821
9,ORGANIC,10/20,712


extração

##**. Extração dos Dados**

In [ ]:
# @title Extração

df = results
df.to_excel('/content/ligacoes_nao_recep.xlsx', index = False)

##**. Conexão ao GCP**

In [ ]:
# @title Conexão ao GCP

serviceAccount = '/content/athenas-364914-4e16df1244dc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

##**. Inserindo Excell no GCP**

In [ ]:
# @title Inserindo Excell no GCP


client = storage.Client.from_service_account_json(json_credentials_path='/content/athenas-364914-4e16df1244dc.json')
bucket = client.get_bucket('saida_modelagem')
object_name_in_gcs_bucket = bucket.blob('ligacoes_nao_recep.xlsx')
object_name_in_gcs_bucket.upload_from_filename('ligacoes_nao_recep.xlsx')
